In [2]:
import sqlite3
from pathlib import Path
import pandas as pd

In [3]:
# Creating the file
Path('../Data/ILINet.db').touch()

In [4]:
# Connecting the file
conn = sqlite3.connect('../Data/ILINet.db')
c = conn.cursor()

In [5]:
# c.execute('''DROP TABLE ilinet''')

In [6]:
# Creating the table
c.execute('''CREATE TABLE ilinet (REGIONTYPE text,REGION text,YEAR int,WEEK int,
PERCENTWEIGHTEDILI float,PERCENTUNWEIGHTEDILI float,
AGE04 int,AGE2549 int,AGE2564 int,AGE524 int,AGE5064 int,AGE65 int,
ILITOTAL int,NUMOFPROVIDERS int,TOTALPATIENTS int,ID int NOT NULL PRIMARY KEY)''')

Working with the data frame to include a primary key manually

In [7]:
# load the data into a Pandas DataFrame
ili_df = pd.read_csv('../Data/ILINet.csv')
ili_df.head()

,REGIONTYPE,REGION,YEAR,WEEK,PERCENTWEIGHTEDILI,PERCENTUNWEIGHTEDILI,AGE04,AGE2549,AGE2564,AGE524,AGE5064,AGE65,ILITOTAL,NUMOFPROVIDERS,TOTALPATIENTS
0,States,Alabama,2013,40,X,1.59044,X,X,X,X,X,X,201,27,12638
1,States,Alaska,2013,40,X,0.6238,X,X,X,X,X,X,13,9,2084
2,States,Arizona,2013,40,X,0.792311,X,X,X,X,X,X,230,59,29029
3,States,Arkansas,2013,40,X,1.63326,X,X,X,X,X,X,66,18,4041
4,States,California,2013,40,X,1.52479,X,X,X,X,X,X,633,137,41514


In [8]:
# creating a list for the primary key
i = 0
id_vals=[]
for i in range(len(ili_df)):
    id_vals.append(i)
    i += 1



In [9]:
# assigning the id into the dataframe
ili_df["ID"] = id_vals

In [10]:
ili_df.head(-5)

,REGIONTYPE,REGION,YEAR,WEEK,PERCENTWEIGHTEDILI,PERCENTUNWEIGHTEDILI,AGE04,AGE2549,AGE2564,AGE524,AGE5064,AGE65,ILITOTAL,NUMOFPROVIDERS,TOTALPATIENTS,ID
0,States,Alabama,2013,40,X,1.59044,X,X,X,X,X,X,201,27,12638,0
1,States,Alaska,2013,40,X,0.6238,X,X,X,X,X,X,13,9,2084,1
2,States,Arizona,2013,40,X,0.792311,X,X,X,X,X,X,230,59,29029,2
3,States,Arkansas,2013,40,X,1.63326,X,X,X,X,X,X,66,18,4041,3
4,States,California,2013,40,X,1.52479,X,X,X,X,X,X,633,137,41514,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8468,States,Utah,2016,39,X,0.530623,X,X,X,X,X,X,59,39,11119,8468
8469,States,Vermont,2016,39,X,1.81564,X,X,X,X,X,X,13,4,716,8469
8470,States,Virginia,2016,39,X,1.54346,X,X,X,X,X,X,1360,136,88114,8470
8471,States,Washington,2016,39,X,0.261976,X,X,X,X,X,X,7,30,2672,8471


In [11]:
# write the data to a sqlite table
ili_df.to_sql('ilinet', conn, if_exists='append', index = False)

8478

In [12]:
# Checking that it loaded correctly
c.execute('''SELECT * FROM ilinet''').fetchone()

('States',
 'Alabama',
 2013,
 40,
 'X',
 1.59044,
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 201,
 27,
 12638,
 0)

In [13]:
# Removing most of the null data
c.execute('''ALTER TABLE ilinet DROP COLUMN REGIONTYPE''')
c.execute('''ALTER TABLE ilinet DROP COLUMN PERCENTWEIGHTEDILI''')
c.execute('''ALTER TABLE ilinet DROP COLUMN AGE04''')
c.execute('''ALTER TABLE ilinet DROP COLUMN AGE2549''')
c.execute('''ALTER TABLE ilinet DROP COLUMN AGE2564''')
c.execute('''ALTER TABLE ilinet DROP COLUMN AGE524''')
c.execute('''ALTER TABLE ilinet DROP COLUMN AGE5064''')
c.execute('''ALTER TABLE ilinet DROP COLUMN AGE65''')



In [14]:
# Checking for a final time
c.execute('''SELECT * FROM ilinet''').fetchone()

('Alabama', 2013, 40, 1.59044, 201, 27, 12638, 0)

Parse the db so we can load the api from it

In [15]:
import numpy as np
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

In [16]:
Base = automap_base()

In [17]:
# Setting up the DB
engine = create_engine("sqlite:///../Data/ILINet.db")

In [18]:
# Reflect the tables
Base.prepare(autoload_with=engine)

In [19]:
# Save the table reference
ILINet = Base.classes.ilinet

In [20]:
# Create the session link
session = Session(engine)
# Querying the data for jsonifying
results = session.query(ILINet.REGION, ILINet.YEAR, ILINet.WEEK, 
                            ILINet.PERCENTUNWEIGHTEDILI, ILINet.ILITOTAL,
                            ILINet.NUMOFPROVIDERS, ILINet.TOTALPATIENTS).all()
# CLose the Session
session.close()

In [21]:
results

[('Alabama', 2013, 40, 1.59044, 201, 27, 12638),
 ('Alaska', 2013, 40, 0.6238, 13, 9, 2084),
 ('Arizona', 2013, 40, 0.792311, 230, 59, 29029),
 ('Arkansas', 2013, 40, 1.63326, 66, 18, 4041),
 ('California', 2013, 40, 1.52479, 633, 137, 41514),
 ('Colorado', 2013, 40, 0.652244, 142, 9, 21771),
 ('Connecticut', 2013, 40, 0.701388, 47, 27, 6701),
 ('Delaware', 2013, 40, 0.49636, 15, 12, 3022),
 ('District of Columbia', 2013, 40, 5.41463, 111, 3, 2050),
 ('Florida', 2013, 40, 'X', 'X', 'X', 'X'),
 ('Georgia', 2013, 40, 1.00692, 380, 41, 37739),
 ('Hawaii', 2013, 40, 1.25261, 18, 16, 1437),
 ('Idaho', 2013, 40, 0.5631, 17, 8, 3019),
 ('Illinois', 2013, 40, 1.41144, 859, 105, 60860),
 ('Indiana', 2013, 40, 0.918555, 90, 38, 9798),
 ('Iowa', 2013, 40, 0.170691, 62, 21, 36323),
 ('Kansas', 2013, 40, 0.333611, 24, 32, 7194),
 ('Kentucky', 2013, 40, 0.242983, 49, 17, 20166),
 ('Louisiana', 2013, 40, 2.02694, 596, 49, 29404),
 ('Maine', 2013, 40, 0.823958, 52, 33, 6311),
 ('Maryland', 2013, 40, 0